### 2.1 PyMC规则与设计模式

#### 一. 父变量与子变量
1. 父变量 : 对其他变量有影响的变量  
 子变量 : 被其他变量影响的变量, 父变量作用于子变量  
2. 一个变量可以即是父变量, 又是子变量
3. 如下代码所示  
  变量_lambda控制着变量_data, 因此_lambda是_data的父变量
 ```python
 import pymc as pm
_lambda = pm.Exponential('lambda',1)
_data = pm.Poisson('data',_lambda)
_data_plus_one = _data + 1
 ```
4. 通过变量的children和parents属性获取变量的父/子变量  
 如下代码发现parents获得_data的父变量的内存地址与_lambda相同  
 ```python
print hex(id(_lambda))  # 0x1a13752990
print _data.parents     # {'mu': <pymc.distributions.Exponential 'lambda' at 0x1a13752990>}
 ```

In [44]:
import pymc as pm
_lambda = pm.Exponential('lambda',1)
_data = pm.Poisson('data',_lambda)
_data_plus_one = _data + 1
pm.Uniform('beats',0,5,size=3)

<pymc.distributions.Uniform 'beats' at 0x1a13833610>

In [48]:
import pymc as pm
import numpy as np
data = np.array([10,5])
fixed_value = pm.Poisson('fixed',1,value=data,observed=True)
print fixed_value.value
fixed_value.random()
print fixed_value.value

[10  5]
[10  5]


In [45]:
pm.Poisson??